In [1]:
import numpy as np
import sympy

from PIL import Image
from imageio import imread

import matplotlib.pyplot as plt

In [39]:
class SV:
    def __init__(self, x_ind):
        self.lower_bound = 0
        self.lower_reach_flg = 1
        self.unkwn_for_reach_lb = np.array([[]] * len(x_ind))
        
        self.upper_bound = None
        self.upper_reach_flg = 0
        self.unkwn_for_reach_ub = np.array([[]] * len(x_ind))
        
    def check_bound(self, A, type_bnd):
        # if from_flg then lower_bound = lower_bound(C -- right)
        # else lower_bound = lower_bound(B -- left -- lower)
        #
        # type_bnd = 'lower' or 'upper'
        
        if type_bnd == "lower":
            sigma = self.lower_bound
            if sigma is None:
                self.unkwn_for_reach_lb = 0
                return
        else:
            sigma = self.upper_bound
            if sigma is None:
                self.unkwn_for_reach_ub = 0
                return
        
        print(type_bnd)
        
        c = A.A[1: , 0 ].reshape((-1, 1))
        r = A.A[0  , 1:].reshape((-1, 1))
        K = A.A[1: , 1:]
        L = c.dot(c.T) + K.dot(K.T) - sigma**2 * np.identity(c.shape[0])
        
        a = np.linalg.det(np.block([[1, c.T], [c, L]]))
        b = np.linalg.det(2 * np.block([[0, r.T.dot(K.T)], [c, L]]))
        c = np.linalg.det(np.block([[r.T.dot(r) - sigma**2, r.T.dot(K.T)], [K.dot(r), L]]))
        
        if abs(a) < 10**(-6):
            if abs(b) < 10**(-6):
                if abs(c) >= 10**(-6):
                    if type_bnd == "lower":
                        self.lower_reach_flg = 0
                    else:
                        self.upper_reach_flg = 0
                    print("No solution, a=b=0, c<>0")
                else:
                    if type_bnd == "lower":
                        self.lower_reach_flg = 1
                    else:
                        self.upper_reach_flg = 1
                    print("x - any, a=b=c=0")
            else:
                if type_bnd == "lower":
                    self.lower_reach_flg = 1
                else:
                    self.upper_reach_flg = 1
                print("x = ", -c/b, " a=0 b<>0")
        else:
            D = b * b - 4 * a * c
            print("We here: a=",a," b=", b, " c=", c, " D=", D)
            if abs(D) < 10**(-6):
                if type_bnd == "lower":
                    self.lower_reach_flg = 1
                else:
                    self.upper_reach_flg = 1
                x = -b/a
                print("x_1 = x_2 =", x)
            elif D < 0:
                if type_bnd == "lower":
                    self.lower_reach_flg = 0
                else:
                    self.upper_reach_flg = 0
                print("No solution, a<>0 D<0")
            else:
                if type_bnd == "lower":
                    self.lower_reach_flg = 1
                else:
                    self.upper_reach_flg = 1
                x_1 = 1/2 * (-b + np.sqrt(D))
                x_2 = 1/2 * (-b - np.sqrt(D))
                print("x_1 = ", x_1, " x_2 = ", x_2)

In [61]:
class Partially_prescribed_matrix:
    def __init__(self, A, x_ind):
        self.A = A.copy()
        self.x_ind = x_ind.copy()
        self.x_ind.sort(key=lambda elem: elem[0] * max(np.array(self.A).shape) + elem[1])
        
        self.sv_fnd_flg = False
        self.sv = [SV(x_ind) for x in range(min(A.shape))]
            
    def print_M(self):
        print("--------PP Matrix--------")
        for i in range(self.A.shape[0]):
            for j in range(self.A.shape[1]):
                if [i, j] in self.x_ind:
                    print("x", end=" ")
                else:
                    print(self.A[i, j], end=" ")
            print(end="\n")
        print("-------------------------")
        
    def print_sv(self):
        if self.sv_fnd_flg == False:
            self.find_sv()
        for i in range(len(self.sv)):
            print("Номер сингулярного числа:", i, "\n  ", self.sv[i].lower_bound, "<", end="")
            if self.sv[i].lower_reach_flg == 1:
                print("= ", end="")
            else:
                print("  ", end="")
            print("s ", end="")
            if self.sv[i].upper_bound is not None:
                print("<", end="")
                if self.sv[i].upper_reach_flg == 1:
                    print("= ", end="")
                else:
                    print("  ", end="")
                print(self.sv[i].upper_bound, end="")
            print(end="\n\n")
    
    # Меняем i-ую строку на нулевую и наоборот
    # Меняем j-ый столбец на нулевой и наоборот
    # elem = (i, j)
    def change_row_column(self, elem):
        A_ = self.A.copy()
        self.A[0] = A_[elem[0]]
        self.A[elem[0]] = A_[0]
        
        A_ = self.A.copy()
        self.A[:, 0] = A_[:, elem[1]]
        self.A[:, elem[1]] = A_[:, 0]       
        
    
    # Перемещаем какой-нибудь неизвестный элемент на (0,0) место
    def move_x(self):
        if len(self.x_ind) == 0:
            return 0
        f_elem = self.x_ind[0].copy()
        self.change_row_column(f_elem)
        for elem in self.x_ind:
            if elem[0] == f_elem[0]:
                elem[0] = 0
            if elem[1] == f_elem[1]:
                elem[1] = 0
        return 1
    
    # Делим матрицу на две, удаляя первую строку или первый столбец
    def div_matrix(self):
        left = Partially_prescribed_matrix(self.A[1:], (np.array(self.x_ind) - [1, 0]).tolist())
        for i in range(np.array(left.A).shape[1]):
            try:
                left.x_ind.remove([-1, i])
            except Exception:
                1
        right = Partially_prescribed_matrix(self.A[:, 1:], (np.array(self.x_ind) - [0, 1]).tolist())
        for i in range(np.array(right.A).shape[0]):
            try:
                right.x_ind.remove([i, -1])
            except Exception:
                1
        return left, right
    
    # Выполняет алгоритм, описанный в курсовой
    def find_sv(self):
        self.sv_fnd_flg = True
        
        for ind in self.x_ind:
            self.A[ind[0], ind[1]] = 0
            
        n = min(np.array(self.A).shape)
        
        if len(self.x_ind) == 0:
            values = np.linalg.svd(self.A)[1].tolist()
            for i in range(len(values)):
                self.sv[i].lower_bound = values[i]
                self.sv[i].lower_reach_flg = 1
                self.sv[i].upper_bound = values[i]
                self.sv[i].upper_reach_flg = 1
            return self.sv
        
        if n == 1:
            self.sv[0].lower_bound = np.sum(self.A)
            self.sv[0].lower_reach_flg = 1
            self.sv[0].unkwn_for_reach_lb = np.array([[0]] * len(self.x_ind))
            return self.sv
        
        self.move_x()
        
        left, right = self.div_matrix()
        l_sv = left.find_sv()
        r_sv = right.find_sv()
        left.print_M()
        left.print_sv()
        right.print_M()
        right.print_sv()
        
        for i in range(min(len(l_sv), len(r_sv))):
            self.sv[i].lower_bound = max(l_sv[i].lower_bound, r_sv[i].lower_bound)
            self.sv[i].check_bound(self, "lower")
            
            if l_sv[i].upper_bound is None:
                self.sv[i + 1].upper_bound = r_sv[i].upper_bound
            elif r_sv[i].upper_bound is None:
                self.sv[i + 1].upper_bound = l_sv[i].upper_bound
            else:
                self.sv[i + 1].upper_bound = min(l_sv[i].upper_bound, r_sv[i].upper_bound)
                
            self.sv[i + 1].check_bound(self, "upper")

        if len(l_sv) > len(r_sv):
            self.sv[len(r_sv)].lower_bound = l_sv[len(r_sv)].lower_bound
            self.sv[len(r_sv)].check_bound(self, "lower")
        elif len(l_sv) < len(r_sv):
            self.sv[len(l_sv)].lower_bound = r_sv[len(l_sv)].lower_bound
            self.sv[len(l_sv)].check_bound(self, "lower")
        else:
            self.sv[len(l_sv)].lower_bound = 0
            self.sv[len(l_sv)].check_bound(self, "lower")
        
        return self.sv

In [62]:
A = np.array([[1, 1], [1, 0]])
x_ind = [[0, 0]]

M = Partially_prescribed_matrix(A, x_ind)
sv = M.find_sv()

M.print_M()
M.print_sv()

--------PP Matrix--------
1 0 
-------------------------
Номер сингулярного числа: 0 
   1.0 <= s <= 1.0

--------PP Matrix--------
1 
0 
-------------------------
Номер сингулярного числа: 0 
   1.0 <= s <= 1.0

lower
We here: a= -1.0  b= 0.0  c= 0.0  D= 0.0
x_1 = x_2 = 0.0
upper
We here: a= -1.0  b= 0.0  c= 0.0  D= 0.0
x_1 = x_2 = 0.0
lower
No solution, a=b=0, c<>0
--------PP Matrix--------
x 1 
1 0 
-------------------------
Номер сингулярного числа: 0 
   1.0 <= s 

Номер сингулярного числа: 1 
   0 <  s <= 1.0



In [51]:
A = np.array([[1, 1, 0], [1, 0, 1]])
x_ind = [[0, 0]]

M = Partially_prescribed_matrix(A, x_ind)
sv = M.find_sv()

M.print_M()
M.print_sv()

lower
We here: a= -1.0000000000000004  b= 0.0  c= 4.4408920985006143e-16  D= 1.7763568394002465e-15
x_1 = x_2 = 0.0
upper
x - any, a=b=c=0
lower
x - any, a=b=c=0
--------PP Matrix--------
x 1 0 
1 0 1 
-------------------------
Номер сингулярного числа: 0 
   1.4142135623730951 <= s 

Номер сингулярного числа: 1 
   1.0 <= s <= 1.0



In [52]:
A = np.array([[1, 1], [1, 0], [0, 1]])
x_ind = [[0, 0]]

M = Partially_prescribed_matrix(A, x_ind)
sv = M.find_sv()

M.print_M()
M.print_sv()

lower
We here: a= 2.0000000000000013  b= 0.0  c= -8.881784197001244e-16  D= 7.105427357601e-15
x_1 = x_2 = -0.0
upper
x - any, a=b=c=0
lower
x - any, a=b=c=0
--------PP Matrix--------
x 1 
1 0 
0 1 
-------------------------
Номер сингулярного числа: 0 
   1.4142135623730951 <= s 

Номер сингулярного числа: 1 
   1.0 <= s <= 1.0



In [53]:
A = np.array([[1, 1, 0], [1, 0, 1], [0, 1, 0]])
x_ind = [[0, 0]]

M = Partially_prescribed_matrix(A, x_ind)
sv = M.find_sv()

M.print_M()
M.print_sv()

lower
We here: a= 1.0000000000000009  b= 0.0  c= -3.944304526105045e-31  D= 1.5777218104420194e-30
x_1 = x_2 = -0.0
upper
We here: a= 1.0000000000000009  b= 0.0  c= -3.944304526105045e-31  D= 1.5777218104420194e-30
x_1 = x_2 = -0.0
lower
No solution, a=b=0, c<>0
upper
No solution, a=b=0, c<>0
lower
We here: a= 1.0  b= 0.0  c= 0.0  D= 0.0
x_1 = x_2 = -0.0
--------PP Matrix--------
x 1 0 
1 0 1 
0 1 0 
-------------------------
Номер сингулярного числа: 0 
   1.4142135623730951 <= s 

Номер сингулярного числа: 1 
   1.0 <  s <= 1.4142135623730951

Номер сингулярного числа: 2 
   0 <= s <  1.0



In [54]:
A = np.array([[1, 1, 0], [1, 0, 1], [0, 1, 0]])
x_ind = [[0, 0], [0, 1]]

M = Partially_prescribed_matrix(A, x_ind)
sv = M.find_sv()

M.print_M()
M.print_sv()

lower
x - any, a=b=c=0
upper
x - any, a=b=c=0
lower
x - any, a=b=c=0
lower
We here: a= 1.0000000000000009  b= 0.0  c= -8.881784197001244e-16  D= 3.552713678800501e-15
x_1 = x_2 = -0.0
upper
We here: a= 1.0000000000000009  b= 0.0  c= -8.881784197001244e-16  D= 3.552713678800501e-15
x_1 = x_2 = -0.0
lower
x - any, a=b=c=0
upper
x - any, a=b=c=0
lower
We here: a= 1.0  b= 0.0  c= 0.0  D= 0.0
x_1 = x_2 = -0.0
--------PP Matrix--------
x x 0 
1 0 1 
0 1 0 
-------------------------
Номер сингулярного числа: 0 
   1.4142135623730951 <= s 

Номер сингулярного числа: 1 
   1.0 <= s <= 1.4142135623730951

Номер сингулярного числа: 2 
   0 <= s <= 1.0



In [60]:
A = np.array([[1, 1], [1, 1]])
x_ind = [[0, 0], [1, 1]]

M = Partially_prescribed_matrix(A, x_ind)
sv = M.find_sv()

M.print_M()
M.print_sv()

--------PP Matrix--------
1 x 
-------------------------
Номер сингулярного числа: 0 
   1 <= s 

--------PP Matrix--------
1 
x 
-------------------------
Номер сингулярного числа: 0 
   1 <= s 

lower
We here: a= -1.0  b= 0.0  c= 0.0  D= 0.0
x_1 = x_2 = 0.0
lower
No solution, a=b=0, c<>0
--------PP Matrix--------
x 1 
1 x 
-------------------------
Номер сингулярного числа: 0 
   1 <= s 

Номер сингулярного числа: 1 
   0 <  s 



In [ ]:
\section{Сегментация изображений} \label{sect3_2}

Чувствительность алгоритма поиска наименьшего ранга к границам и резкой смене цвета, позволяет рассмотреть также задачу сегментации. Смысл поставленной задачи заключается в том, чтобы по полностью известному изображению получить некоторые границы объектов на нем. Идея применения метода заключается в том, чтобы вырезать каждый второй пиксель на известной фотографии и пройти с маской 2 на 2 по ней. Когда алгоритм не сможет определить означивание